##I - Introduction

#####2018 a été une année décisive dans le NLP. L'apprentissage par transfert, en particulier des modèles tels que ELMO d'Allen AI, Open-GPT d'OpenAI et BERT de Google, a fourni au reste de la communauté NLP des modèles pré-entraînés qui pourraient facilement avec moins de données et moins de temps de calcul être affinés et mis en œuvre pour produire des résultats qui dépasse the state of the art. Dans ce projet,nous allons montrer comment utiliser BERT avec la bibliothèque PyTorch pour affiner rapidement et efficacement un modèle de classification des logs de Thunderbird supercomputer(contient 211,212,192 logs mais nous allons uiliser 10K logs), où notre objectif sera de détecter les anomalies dans des futurs logs.


- **Dataset** - nous allons utiliser une DT open Source Thunderbird mais il est sous forme d'un Filelog donc on avait besoin d'utiliser excel(On peut utiliser Python mais excel reste simple) pour insérer les logs dans des dataframes.

- **Objectif** - developper une solution qui permet de detecter les anomalies dans les logs.

- **Methodologie** - nous allons considerer comme nous avons un probleme de text classification et construire un deep learning model pour attiendre l'objectif.

#II- Different Models 
Il existe plusieurs types de modèles qui pourraient être utilisés pour créer la solution pour la classification de texte. Quelques exemples sont :


*   **1D- Conv Net** : 
les CNN peuvent être utilisés pour la classification des textes. **Avantage**: ils sont plus rapides à s'entraîner. En fait, un modèle CNN pourrait atteindre une précision décente **Disavantage**: ils ne parviennent pas à capturer les dépendances à long terme dans le texte et ne capturent pas les informations séquentielles dans le texte.

*   **Modèles basés sur RNN (LSTM, GRU)** : **Avantage**: ils peuvent capturer la nature séquentielle d'un texte. **Inconvénient** : Plus lent à s'entraîner.

* **Modèles basés sur des transformers (BERT, GPT2)** -
Les modèles basés sur des transformers exploite plusieurs unités Transformer et un mécanisme d'attention à multihead. L'avantage est qu'ils se concentrent uniquement sur le mécanisme d'attention. 

####Dans ce projet nous allons focaliser sur un model basé sur des transformers(encoder partie) qui est le model Bert 


# III- Bert For anomaly detection

### 1. Importing libraries and modules

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 28.5MB/s 
     |████████████████████████████████| 3.3MB 46.8MB/s 
     |████████████████████████████████| 901kB 46.7MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import re
from google.colab import drive
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
from sklearn.utils.class_weight import compute_class_weight


###2. Loading Data

####**Importing files from Google Drive in Colab**
notre dataset est stockée dans Google Drive donc nous avons besoin de 
lier notre compte Google Drive avec notre notebook.
1. La première étape consiste à monter notre Google Drive en exécutant le code en dessous.
2.  nous obtenons le code d'autorisation en nous connectant à notre compte Google.
3.   nous collons le code d'autorisation et nous appuyons sur Entrée.

In [ ]:
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
PATH_normal=r'/gdrive/My Drive/thundn.xlsx'
PATH_abnormal=r'/gdrive/My Drive/thunda.xlsx'

In [ ]:
data_normal = pd.read_excel(PATH_normal)
data_abnormal = pd.read_excel(PATH_abnormal)

In [ ]:
data_normal.to_csv(r'/gdrive/My Drive/tnnormall.csv')
data_abnormal.to_csv(r'/gdrive/My Drive/tnabnormall.csv')

In [ ]:
df_normal = pd.read_csv(r'/gdrive/My Drive/tnnormall.csv')
df_abnormal = pd.read_csv(r'/gdrive/My Drive/tnabnormall.csv')

###3. Data Preprocessing

In [ ]:
df_normal=df_normal[['log']]

In [ ]:
df_abnormal=df_abnormal[['log']]

In [ ]:
df_normal['label']=0
df_abnormal['label']=1

In [ ]:
#concatinate
df=pd.concat([df_normal,df_abnormal])
# Shuffle the data
df = shuffle(df).reset_index(drop=True)
df.sample(10)

,log,label
5125,VAPI 1135043633 2005.12.19 an690 Dec 19 17:53:...,1
10023,- 1133653376 2005.12.03 tbird-sm1 Dec 3 15:42:...,0
8567,- 1133651756 2005.12.03 tbird-sm1 Dec 3 15:15:...,0
5637,VAPI 1135049860 2006.01.20 an690 Jan 19 19:37:...,1
8669,- 1131993970 2005.11.14 badmin3 Nov 14 10:46:1...,0
3959,- 1133654461 2005.12.03 bn1017 Dec 3 16:01:01 ...,0
3840,- 1131994499 2005.11.14 an256 Nov 14 10:54:59 ...,0
9889,- 1133653603 2005.12.03 tbird-sm1 Dec 3 15:46:...,0
4871,- 1131994704 2005.11.14 an81 Nov 14 10:58:24 a...,0
1267,- 1131994130 2005.11.14 an507 Nov 14 10:48:50 ...,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10290 entries, 0 to 10289
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   log     10290 non-null  object
 1   label   10290 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 160.9+ KB


In [ ]:
def clean_data(log):
    log = re.sub("'", "", log)
    log = re.sub("-", "", log)
    log = re.sub("(\\W)+", " ", log)
    log = log.lower()
    return log

df['log'] = df['log'].apply(clean_data)

* Split the dataset into train(80%, validation 10% and test sets 10%)

In [ ]:
train_text, temp_text, train_labels, temp_labels = train_test_split(df['log'], df['label'], 
                                                                    random_state=2018, 
                                                                    test_size=0.2, 
                                                                    stratify=df['label'])

# val set & test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [ ]:
# specify GPU
device = torch.device("cuda")

###4. Tokenization

* Bert accepte un type spécifique des inputs pour y répondre nous sommes tenus de:
1. Ajouter des tokens spéciaux au début[CLS] et à la fin[SEP] de chaque log.
2. Compléter et tronquer toutes les logs à une seule longueur constante "padding".
3. Différencier explicitement les vrais tokens des tokens de remplissage avec le "attention mask" 1 vrai 0 vide.

PS: l'output du dernier transformer(12eme), seul le premier embeddage (correspondant au token [CLS]) est utilisé par le classifieur.

In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

* Mesurer la longueur du plus long log

In [ ]:
max_len = 0
for sent in df.log:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  114


* fixer la longueur des logs

In [ ]:
max_seq_len = max_len

* Tokenize DataSet

In [ ]:
# tokenize & encode sequences training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
    max_length = max_seq_len,
    pad_to_max_length=True, # Pad & truncate all sentences.
    truncation=True,
    return_token_type_ids=False
)

# tokenize & encode sequences validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize & encode sequences test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


###5. Convert Data to Tensors(PyTorch Data Types)

Notre modèle attend des tenseurs PyTorch plutôt que numpy.ndarrays, donc convertissons toutes nos variables.

In [ ]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

###6. Create DataLoaders

Nous allons également créer un itérateur pour notre ensemble de données à l'aide de la classe Torch DataLoader. Cela permet d'économiser de la mémoire pendant l'entraînement car, contrairement à une boucle for, avec un itérateur, l'ensemble de données n'a pas besoin d'être chargé en mémoire.

In [ ]:
#define a batch size
batch_size = 16

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

###7. Define Model Architecture

In [ ]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


* Freeze BERT Parameters

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      # apply softmax activation
      x = self.softmax(x)

      return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# Tell pytorch to run this model on the GPU.
model = model.to(device)

Maintenant que notre modèle est chargé, nous devons récupérer les hyperparamètres d'entraînement.
- Batch size: 16 (nous en avons choisi 16 lors de la création du DataLoaders).
- Learning rate (Adam):nous utiliserons 2e-5.
- Nombre d'époques:nous en utiliserons 4.

In [ ]:
# define the optimizer
optimizer = AdamW(model.parameters(), lr =2e-5)

In [ ]:
#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

[0.52932099 9.02631579]


In [ ]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 4

###8. Fine-Tune BERT

In [ ]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      #elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)
      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

###9. Model Training

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 4
  Batch    50  of    515.
  Batch   100  of    515.
  Batch   150  of    515.
  Batch   200  of    515.
  Batch   250  of    515.
  Batch   300  of    515.
  Batch   350  of    515.
  Batch   400  of    515.
  Batch   450  of    515.
  Batch   500  of    515.

Evaluating...
  Batch    50  of     65.

Training Loss: 0.561
Validation Loss: 0.377

 Epoch 2 / 4
  Batch    50  of    515.
  Batch   100  of    515.
  Batch   150  of    515.
  Batch   200  of    515.
  Batch   250  of    515.
  Batch   300  of    515.
  Batch   350  of    515.
  Batch   400  of    515.
  Batch   450  of    515.
  Batch   500  of    515.

Evaluating...
  Batch    50  of     65.

Training Loss: 0.372
Validation Loss: 0.201

 Epoch 3 / 4
  Batch    50  of    515.
  Batch   100  of    515.
  Batch   150  of    515.
  Batch   200  of    515.
  Batch   250  of    515.
  Batch   300  of    515.
  Batch   350  of    515.
  Batch   400  of    515.
  Batch   450  of    515.
  Batch   500  of    515.

Evalu

In [ ]:
# get predictions for test data
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()

###10. Get Predictions for Test Data

In [ ]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       972
           1       0.97      1.00      0.98        57

    accuracy                           1.00      1029
   macro avg       0.98      1.00      0.99      1029
weighted avg       1.00      1.00      1.00      1029

